<a href="https://colab.research.google.com/github/stevejj4/Data-Cleaning/blob/main/SUN_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default

# Authorize Google Colab
auth.authenticate_user()

# Credentials for Google Sheets
creds, _ = default()
gc = gspread.authorize(creds)

# Load the Google Sheet
spreadsheet = gc.open('SUN_Dashboard')

# Check if the sheet 'transformed' exists and create it if not
try:
    transformed_sheet = spreadsheet.worksheet('transformed')
except gspread.exceptions.WorksheetNotFound:
    transformed_sheet = spreadsheet.add_worksheet(title='transformed', rows="1000", cols="20")

# Load the original data
worksheet = spreadsheet.sheet1
rows = worksheet.get_all_values()

# Create DataFrame from the sheet data
df = pd.DataFrame(rows)
df.columns = df.iloc[0]  # Set the first row as column names
df = df.iloc[1:]  # Remove the first row


In [2]:
df.shape

(68, 21)

In [3]:
# Reshape the data from wide to long format
value_vars = ['Contribution', 'Compensation', 'Deaths']
months = ['Jan', 'Feb', 'March', 'April', 'May']

# Initialize an empty list to store the reshaped data
reshaped_data = []

# Iterate through each row in the DataFrame
for _, row in df.iterrows():
    for month in months:
        reshaped_data.append({
            'PAYBILL NO': row['PAYBILL NO'],
            'SUB COUNTY': row['SUB COUNTY'],
            'REGION': row['REGION'],
            'Cluster': row['Cluster'],
            'County': row['County'],
            'Site': row['Site'],
            'Month': month,
            'Contribution': row[f'{month}_Contribution'],
            'Compensation': row[f'{month}_Compensation'],
            'Deaths_per_Month': row[f'{month}_deaths']
        })

# Create a new DataFrame from the reshaped data
reshaped_df = pd.DataFrame(reshaped_data)

# Convert the DataFrame to a list of lists
reshaped_data_list = [reshaped_df.columns.values.tolist()] + reshaped_df.values.tolist()

# Clear the 'transformed' sheet
transformed_sheet.clear()

# Write the reshaped DataFrame to the 'transformed' sheet
transformed_sheet.update('A1', reshaped_data_list)

# Print confirmation
print('Data successfully loaded into the transformed sheet.')


<ipython-input-3-c3032ec87045>:34: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  transformed_sheet.update('A1', reshaped_data_list)


Data successfully loaded into the transformed sheet.


In [4]:
reshaped_df.shape

(340, 10)

In [8]:
reshaped_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 340 entries, 0 to 339
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   PAYBILL NO        340 non-null    object
 1   SUB COUNTY        340 non-null    object
 2   REGION            340 non-null    object
 3   Cluster           340 non-null    object
 4   County            340 non-null    object
 5   Site              340 non-null    object
 6   Month             340 non-null    object
 7   Contribution      340 non-null    object
 8   Compensation      340 non-null    object
 9   Deaths_per_Month  340 non-null    object
dtypes: object(10)
memory usage: 26.7+ KB


In [9]:
reshaped_df.describe()

,PAYBILL NO,SUB COUNTY,REGION,Cluster,County,Site,Month,Contribution,Compensation,Deaths_per_Month
count,340,340,340,340,340,340,340,340,340,340
unique,68,68,9,4,27,3,5,335,39,31
top,4068299,KIBERA,Nyanza,,Nairobi,Sub County,Jan,"167,400.00",0.00,0.00
freq,5,5,85,260,80,230,68,2,129,102
